# Settings

In [ ]:
!pip install unidecode
import torch
import torch.nn as nn
from torch.autograd import Variable
import unidecode
import string
import random
import re
import time, math

     |████████████████████████████████| 245kB 8.3MB/s eta 0:00:01


# Hyperparameters

In [ ]:
num_epochs = 10000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size = 1
num_layers = 2
lr = 0.002


# Data

In [ ]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [ ]:
file = unidecode.unidecode(open('/content/drive/MyDrive/Colab Notebooks/input.txt').read())
file_len = len(file)
print('file_len = ', file_len)


file_len =  1115394


# Functions for text processing

In [ ]:
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]

print(random_chunk())

tian field,
Streaming the ensign of the Christian cross
Against black pagans, Turks, and Saracens:
And toil'd with works of war, retired himself
To Italy; and there at Venice gave
His body to that plea


In [ ]:
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
    tensor[c] = all_characters.index(string[c])
  return Variable(tensor).cuda()

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15], device='cuda:0')


In [ ]:
def random_training_set():
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

# RNN Class

In [ ]:
class RNN(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layer=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    self.embedding_size = embedding_size

    self.encoder = nn.Embedding(input_size, embedding_size)
    self.rnn = nn.GRU(embedding_size, hidden_size, num_layers)
    self.decoder = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden):
    out = self.encoder(input.view(1, -1))
    out, hidden = self.rnn(out, hidden)
    out = self.decoder(out.view(batch_size, -1))

    return out, hidden

  def init_hidden(self):
    hidden = Variable(torch.zeros(num_layers, batch_size, hidden_size)).cuda()

    return hidden

model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers).cuda()

In [ ]:
inp = char_tensor("A")
print(inp)
hidden = model.init_hidden()
print(hidden.size())

out, hidden = model(inp, hidden)
print(out.size())

tensor([36], device='cuda:0')
torch.Size([2, 1, 100])
torch.Size([1, 100])


# Loss & Optimizer

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

# Test function

In [ ]:
def test():
  start_str = "b"
  inp = char_tensor(start_str)
  hidden = model.init_hidden()
  x = inp

  print(start_str, end="")
  
  for i in range(200):
    output, hidden = model(x, hidden)

    output_dist = output.data.view(-1).div(0.8).exp()
    top_i = torch.multinomial(output_dist, 1)[0]
    predicted_char = all_characters[top_i]

    print(predicted_char, end="")
    
    x = char_tensor(predicted_char)

# Train

In [ ]:
for i in range(num_epochs):
  total = char_tensor(random_chunk())
  inp = total[:-1]
  label = total[1:]
  hidden = model.init_hidden()

  loss = 0
  optimizer.zero_grad()
  for j in range(chunk_len - 1):
    x = inp[j]
    y_ = label[j].unsqueeze(0).type(torch.LongTensor).cuda()
    y, hidden = model(x, hidden)
    loss += loss_func(y, y_)

  loss.backward()
  optimizer.step()

  if i % 100 == 0:
    print("\n", loss/chunk_len, "\n")
    test()
    print("\n\n")


 tensor(4.5870, device='cuda:0', grad_fn=<DivBackward0>) 

'@)5*AA\i.uD



 tensor(2.4408, device='cuda:0', grad_fn=<DivBackward0>) 

b-!L":S-he fhe thee nalked is sotherle igre he gaet,
Egor lery thused sam sorer ase, thisete, ars rare pusar sand yot thoun thelem
IAnd arer.

E	}NEI:
Tharl thralmessh ther,
REy ofrve tor talleln maak 



 tensor(2.2928, device='cuda:0', grad_fn=<DivBackward0>) 

brogzortere ey in misline sull su noste math sall the an mabce lear!
Bath pranlagt pt your and san id is:
Wher a cinssat are,
The thas jowwre an.
Afith have prarh'd thatF your hour node tyar yous and 



 tensor(2.0368, device='cuda:0', grad_fn=<DivBackward0>) 

bde the whe bronent to mungon ould he home sin in, there I tue the houp ind? I the she the his in of gith made
She mano of menind bim is and wirass this cnallowbporgherto ther.

SELAVUSI PERILGRANE:
Ha



 tensor(2.0285, device='cuda:0', grad_fn=<DivBackward0>) 

blemes mone; here I to it what chear to tleis have panken, eptill nequade

KeyboardInterrupt: ignored